In [0]:
%run ./Classroom-Setup-Common

In [0]:

# # 
# import os

# def create_objects_demo_scdtype2(catalog: str, schema: str, source_volume: str) -> str:
#     """
#     Sets up a Databricks lab environment for a workshop by creating a catalog, schema, 
#     and staging volume if they do not already exist.

#     Args:
#         catalog (str): The Unity Catalog name to use for the lab.
#         schema (str): The schema name to create and use within the catalog. Will create three schemas (bronze, silver, gold).
#         source_volume (str): The name of the source volume for lab data.

#     Returns:
#         str: The full path to the created lab source files volume.

#     Raises:
#         ValueError: If any of the input arguments are empty or invalid.
#         Exception: If Spark execution fails during setup.
#     """
     
#     try:
#         ## Bronze, silver, gold
#         bronze_schema = f'{schema}_1_bronze'
#         print(f'Creating schema {catalog}.{bronze_schema}...')
#         spark.sql(f'CREATE SCHEMA IF NOT EXISTS {catalog}.{bronze_schema}')

#         silver_schema = f'{schema}_2_silver'
#         print(f'Creating schema {catalog}.{silver_schema}...')
#         spark.sql(f'CREATE SCHEMA IF NOT EXISTS {catalog}.{silver_schema}')

#         gold_schema = f'{schema}_3_gold'
#         print(f'Creating schema {catalog}.{gold_schema}...')
#         spark.sql(f'CREATE SCHEMA IF NOT EXISTS {catalog}.{gold_schema}')

#         ## Create staging volume in the bronze schema
#         create_volume = f'{catalog}.{bronze_schema}.{source_volume}'
#         print(f'Creating volume {create_volume}...')
#         spark.sql(f'CREATE VOLUME IF NOT EXISTS {create_volume}')

#         ## Create and store the path to their volume and return
#         volume_path = f'/Volumes/{catalog}/{bronze_schema}/{source_volume}'


#         print('\n---------- Schema and volume setup complete ----------\n')
#         # Return volume path to use in demonstration

#         return volume_path


#     except Exception as e:
#         print("⚠️ Error during setup_lab_scdtype2 execution.")
#         print(f"Details: {e}")
#         raise

In [0]:
# # --------------------------------------------------------
# # DO NOT MODIFY BELOW
# # --------------------------------------------------------
check_required_vars("your_marketplace_share_catalog_name","my_catalog")

def auto_cdc_demo_setup(
    my_catalog: str, 
    marketplace_catalog: str, 
    schema: str,
    source_volume: str,
    reset_volume = False
):

    ## Set the default catalog
    r = spark.sql(f"USE CATALOG {my_catalog}")

    ## Create the user schemas and volume. Bronze, silver, gold
    bronze_schema = f'{schema}_1_bronze'
    print(f'Creating schema {my_catalog}.{bronze_schema}...')
    spark.sql(f'CREATE SCHEMA IF NOT EXISTS {my_catalog}.{bronze_schema}')

    silver_schema = f'{schema}_2_silver'
    print(f'Creating schema {my_catalog}.{silver_schema}...')
    spark.sql(f'CREATE SCHEMA IF NOT EXISTS {my_catalog}.{silver_schema}')

    gold_schema = f'{schema}_3_gold'
    print(f'Creating schema {my_catalog}.{gold_schema}...')
    spark.sql(f'CREATE SCHEMA IF NOT EXISTS {my_catalog}.{gold_schema}')

    ## Create staging volume in the bronze schema
    create_volume = f'{my_catalog}.{bronze_schema}.{source_volume}'
    print(f'Creating volume {create_volume}...')
    spark.sql(f'CREATE VOLUME IF NOT EXISTS {create_volume}')

    ## Create and store the path to their volume and return
    my_vol_path = f'/Volumes/{my_catalog}/{bronze_schema}/{source_volume}'

    ## Create a SQL variable to avoid using spark.sql in queries
    spark.sql(f"DECLARE OR REPLACE VARIABLE my_vol_path STRING DEFAULT '{my_vol_path}'")

    print('\n---------- Schema and volume setup complete ----------\n')


    ## Delete all files in the labuser's volume to reset the class if necessary. Otherwise does nothing.
    if reset_volume == True:
        print('Reset volume by deleting files')
        delete_source_files(f'/Volumes/{my_catalog}/{schema}_1_bronze/customer_source_files/')

    # Copy 1 file into the user's volume
    copy_files(
        copy_from = f'/Volumes/{marketplace_catalog}/v01/retail-pipeline/customers/stream_json', 
        copy_to = f'/Volumes/{my_catalog}/{schema}_1_bronze/customer_source_files', 
        n = 1
    )

    compute_validation(recommend_dbr_classic_version=None, recommended_serverless_version=4)

    ## Setup complete displayed
    setup_complete_msg()

    display_config_values([
        ('Your Marketplace Share Catalog', marketplace_catalog),
        ('Your Catalog', my_catalog),
        ('Your Schemas', f"{bronze_schema}, {silver_schema}, {gold_schema}"),
        ('Your Volume Path', my_vol_path)
    ])

    ## Return this to path to to use in the notebook
    return my_vol_path